In [11]:
#importing necessary libraries
import numpy as np
import cv2
import imutils
from imutils import contours

In [12]:
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1} #Change the answer key as per requirment

In [15]:
#load the image of the omr sheet
image = cv2.imread('omr_sheet.png')

#convert it to greyscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

#Apply Gaussian blur filter of size 5 to remove noise
blurred = cv2.GaussianBlur(gray, (5, 5), 0)

#perform edge detection using canny
edged = cv2.Canny(blurred, 75, 200)

cv2.imshow("edge", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [16]:
# find contours in the edge map, then initialize
# the contour that corresponds to the omr sheet

cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)

cnts = imutils.grab_contours(cnts)
docCnt = None

# ensure that at least one contour was found
if len(cnts) > 0:
    # sort the contours according to their size in
    # descending order
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # loop over the sorted contours
    for c in cnts:
        # approximate the contour i.e, convert the contour into a basic shape like a rectangle
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        # if our approximated contour has four points,
        # then we can assume we have found the omr sheet
        if len(approx) == 4:
            docCnt = approx
            break

In [17]:
#Crop original image and grayscale image to obtain a top-down
# birds eye view of the paper

paper = image[ docCnt[0][0][1]:docCnt[1][0][1],docCnt[0][0][0]:docCnt[2][0][0]]
warped = gray[ docCnt[0][0][1]:docCnt[1][0][1],docCnt[0][0][0]:docCnt[2][0][0]]

cv2.imshow("bird view", warped)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [18]:
# apply Otsu's thresholding method to binarize the warped
# piece of paper
thresh = cv2.threshold(warped, 0, 255,
    cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

cv2.imshow("bird view", thresh)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [19]:
# find contours in the thresholded image, then initialize
# the list of contours that correspond to questions
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []

# loop over the contours
for c in cnts:
    # compute the bounding box of the contour
    (x, y, w, h) = cv2.boundingRect(c)
    
    # Use bounding box to derive the aspect ratio
    ar = w / float(h)

    # in order to label the contour as a answering dot, region
    # should be sufficiently wide, sufficiently tall, and
    # have an aspect ratio approximately equal to 1(circle)
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

# sort the question contours top-to-bottom 
questionCnts = contours.sort_contours(questionCnts,
    method="top-to-bottom")[0]



In [21]:
# Initialize the total number of correct answers
correct = 0

# each question has 5 possible answers, to loop over the
# question in batches of 5
paper1 = paper
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):  #array([ 0,  5, 10, 15, 20])
    # sort the contours for the current question from
    # left to right, then initialize the index of the
    # bubbled answer
    cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
    bubbled = None

    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # construct a mask that reveals only the current
        # "bubble" for the question
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        # apply the mask to the thresholded image, then
        # count the number of non-zero pixels in the
        # bubble area
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)
        
        # if the current total has a larger number of total
        # non-zero pixels, then we are examining the currently
        # bubbled-in answer
        if  bubbled != None and total>300:
            break #multiple bubbles bubbled
        if bubbled is None and total > 300:
            bubbled = (total, j)

    # initialize the contour color and the index of the
    # *correct* answer
    color = (0, 0, 255)
    k = ANSWER_KEY[q]

    # check to see if the bubbled answer is correct
    if bubbled != None:    #ruling out no bubbled case
        if k == bubbled[1]:
            color = (0, 255, 0)
            correct += 1

    # draw the outline of the correct answer on the test
    cv2.drawContours(paper1, [cnts[k]], -1, color, 3)

# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper1, "{:.2f}%".format(score), (10, 30),
	cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper1)
cv2.waitKey(0)
cv2.destroyAllWindows()

[INFO] score: 80.00%
